In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import interact
import warnings 
warnings.filterwarnings('ignore')
sns.set_theme(palette='dark')
import optuna
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
import pickle
import joblib
from joblib import Parallel,delayed

In [2]:
df = pd.read_csv('train_CloudCondition.csv')

In [3]:
temp = df.dropna(axis=0)
df = temp

In [4]:
df['Temperature (C)'] = np.where(df['Temperature (C)']=='-','-10',df['Temperature (C)'])
lis = df['Temperature (C)']
new = []
for i in lis:
    new.append(float(i))
df['Temperature (C)'] = new

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70939 entries, 0 to 71427
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Day                         70939 non-null  int64  
 1   Cloud_Condition             70939 non-null  object 
 2   Rain_OR_SNOW                70939 non-null  object 
 3   Temperature (C)             70939 non-null  float64
 4   Apparent Temperature (C)    70939 non-null  float64
 5   Humidity                    70939 non-null  float64
 6   Wind Speed (km/h)           70939 non-null  float64
 7   Wind Bearing (degrees)      70939 non-null  float64
 8   Visibility (km)             70939 non-null  float64
 9   Pressure (millibars)        70939 non-null  float64
 10  Condensation                70939 non-null  object 
 11  Solar irradiance intensity  70939 non-null  int64  
dtypes: float64(7), int64(2), object(3)
memory usage: 7.0+ MB


In [7]:
encoder = LabelEncoder()
df['Cloud_Condition'] = encoder.fit_transform(df['Cloud_Condition'])

In [8]:
x = df.drop(['Day','Cloud_Condition'],axis=1)
y = df['Cloud_Condition']
scale = StandardScaler()
x[['Apparent Temperature (C)','Humidity','Wind Speed (km/h)','Wind Bearing (degrees)',
                         'Visibility (km)',
                        'Pressure (millibars)','Solar irradiance intensity','Temperature (C)']]=scale.fit_transform(x[['Apparent Temperature (C)','Humidity','Wind Speed (km/h)','Wind Bearing (degrees)',
                         'Visibility (km)',
                        'Pressure (millibars)','Solar irradiance intensity','Temperature (C)']])
x = pd.get_dummies(x)

In [9]:
x.head()

,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Pressure (millibars),Solar irradiance intensity,Rain_OR_SNOW_rain,Rain_OR_SNOW_snow,Condensation_Dry,Condensation_Fog,Condensation_Frost,Condensation_Mist
0,-1.245416,-1.286016,-1.266865,-0.784853,-1.072012,-0.817948,-1.104739,-1.254498,1,0,0,0,1,0
1,0.342674,-0.028980,1.196975,-1.271107,1.077728,-1.225382,0.960256,0.285282,1,0,0,0,1,0
2,1.363589,-0.919381,0.907911,0.025570,-1.419055,-0.003078,-1.399738,1.265769,1,0,1,0,0,0
3,1.193436,1.594693,-0.849799,-0.892909,-0.474326,-1.021665,-0.514740,1.106958,0,1,1,0,0,0
4,1.023284,1.280434,-0.019272,1.700443,-1.149132,1.422942,-1.178489,0.975766,0,1,0,1,0,0


In [10]:

rus = RandomOverSampler(random_state=47,sampling_strategy='minority')

x_rus1, y_rus1 = rus.fit_resample(x, y)

print('Original dataset shape', y.shape)
print('Resample dataset shape', y_rus1.shape)

Original dataset shape (70939,)
Resample dataset shape (92799,)


# XGboost

In [37]:
x_train,x_test,y_train,y_test = train_test_split(x_rus1,y_rus1,random_state=47,test_size=0.3)

xgb = XGBClassifier()
xgb.fit(x_train,y_train)
train_pred = xgb.predict(x_train)
test_pred = xgb.predict(x_test)
print()
print(accuracy_score(train_pred,y_train))
print('-x-'*20)
print(accuracy_score(test_pred,y_test))


0.6683446481626872
-x--x--x--x--x--x--x--x--x--x--x--x--x--x--x--x--x--x--x--x-
0.46663074712643676


def objective(trial):
    n_estimators = trial.suggest_int('n_estimators',10,300)
    max_depth = trial.suggest_int('max_depth',2,7)
    learning_rate = trial.suggest_float('learning_rate',0.01, 0.3, log=True)
    subsample = trial.suggest_float('subsample',0.5, 1, log=True)
    colsample_bytree = trial.suggest_float('colsample_bytree',0.5, 1, log=True)
    reg_lambda = trial.suggest_float('reg_lambda',0.1, 10, log=True)
    reg_alpha = trial.suggest_float('reg_alpha',0.1, 10, log=True)

    objective = trial.suggest_categorical('objective',['softmax','softprob'])
    eval_metric = trial.suggest_categorical('eval_metric',['merror','mlogloss'])
    model = XGBClassifier(n_estimators=n_estimators,max_depth=max_depth,learning_rate=learning_rate,
                         subsample=subsample,colsample_bytree=colsample_bytree,reg_lambda=reg_lambda,
                         reg_alpha=reg_alpha,objective=objective,eval_metric=eval_metric)
    cv_score = np.mean(cross_val_score(model,x_train,y_train,scoring = 'f1',cv=5))
    return cv_score
3. Create a study object and optimize the objective function.
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=3)

# GradientBoost

In [38]:
grd = GradientBoostingClassifier()
grd.fit(x_train,y_train)
train_pred = grd.predict(x_train)
test_pred = grd.predict(x_test)
print()
print(accuracy_score(train_pred,y_train))
print('-x-'*20)
print(accuracy_score(test_pred,y_test))


0.5066272571929986
-x--x--x--x--x--x--x--x--x--x--x--x--x--x--x--x--x--x--x--x-
0.4807830459770115


def objective(trial):
    n_estimators = trial.suggest_int('n_estimators',10,300)
    learning_rate = trial.suggest_float('learning_rate',1e-7, 2, log=True)
    min_impurity_decrease = trial.suggest_float('min_impurity_decrease',0.19, 2, log=True)
    subsample = trial.suggest_float('subsample',1e-6, 1, log=True)
    min_samples_split = trial.suggest_float('min_samples_split',1e-6, 0.99, log=True)
    min_weight_fraction_leaf = trial.suggest_float('min_weight_fraction_leaf',1e-6, 0.49, log=True)
    ccp_alpha = trial.suggest_float('ccp_alpha',0.1, 5, log=True)
    random_state= trial.suggest_int('random_state',1,100)
    criterion = trial.suggest_categorical('criterion',['friedman_mse','squared_error','mse'])
    max_depth = trial.suggest_int('max_depth',2,7)
    max_leaf_nodes = trial.suggest_int('max_leaf_nodes',2,5)
    model = GradientBoostingClassifier(n_estimators=n_estimators,learning_rate=learning_rate,
                                       min_impurity_decrease=min_impurity_decrease,
                                      subsample=subsample,min_samples_split=min_samples_split,
                                       min_weight_fraction_leaf=min_weight_fraction_leaf,
                                      ccp_alpha=ccp_alpha,random_state=random_state,criterion=criterion,
                                      max_depth=max_depth,max_leaf_nodes=max_leaf_nodes,warm_start=True,validation_fraction=0.2)
    score = np.mean(cross_val_score(model,x_train,y_train,scoring='f1_macro'))
    return score
3. Create a study object and optimize the objective function.
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

# MLP

In [39]:
mlp = MLPClassifier(random_state=47, max_iter=300)
mlp.fit(x_train,y_train)
train_pred = mlp.predict(x_train)
test_pred = mlp.predict(x_test)
print()
print(accuracy_score(train_pred,y_train))
print('-x-'*20)
print(accuracy_score(test_pred,y_test))


0.4936498406687295
-x--x--x--x--x--x--x--x--x--x--x--x--x--x--x--x--x--x--x--x-
0.47248563218390804


In [29]:
classifiers = [['Neural Network :', MLPClassifier(max_iter = 1000)],
               ['LogisticRegression :', LogisticRegression(max_iter = 1000)],
               ['ExtraTreesClassifier :', ExtraTreesClassifier()],
               ['DecisionTree :',DecisionTreeClassifier()],
               ['RandomForest :',RandomForestClassifier()], 
               ['Naive Bayes :', GaussianNB()],
               ['KNeighbours :', KNeighborsClassifier()],
               ['SVM :', SVC()],
               ['AdaBoostClassifier :', AdaBoostClassifier()],
               ['GradientBoostingClassifier: ', GradientBoostingClassifier()],
               ['XGB :', XGBClassifier()],
               ['CatBoost :', CatBoostClassifier(logging_level='Silent')]]

predictions_df = pd.DataFrame()
predictions_df['action'] = y_test

for name,classifier in classifiers:
    classifier = classifier
    classifier.fit(x_train,y_train)
    predictions = classifier.predict(x_test)
    predictions_df[name.strip(" :")] = predictions
    print(name, accuracy_score(y_test, predictions))

Neural Network : 0.46979166666666666
LogisticRegression : 0.4844109195402299
ExtraTreesClassifier : 0.46404454022988506
DecisionTree : 0.41052442528735633
RandomForest : 0.4687859195402299
Naive Bayes : 0.254992816091954
KNeighbours : 0.42945402298850577
SVM : 0.4849137931034483
AdaBoostClassifier : 0.23617097701149425
GradientBoostingClassifier:  0.48085488505747126
XGB : 0.46663074712643676
CatBoost : 0.47079741379310347


In [40]:
x_train,x_test,y_train,y_test = train_test_split(x_rus1,y_rus1,random_state=47,test_size=0.2)

In [68]:
#joblib.dump(xgb,'xgboost.pkl')
#joblib.dump(vote,'votting.pkl')
#model = joblib.load('votting_model.pkl')

['votting.pkl']

In [12]:
xgb = joblib.load('xgboost.pkl')
grd = joblib.load('gredient.pkl')

In [65]:
logi = LogisticRegression()
mlp = MLPClassifier(random_state=47, max_iter=300)
svm = SVC()
cat = CatBoostClassifier(logging_level='Silent')

In [66]:
x_train,x_test,y_train,y_test = train_test_split(x_rus1,y_rus1,random_state=81,test_size=0.3)
vote = VotingClassifier(estimators=[('xgb', xgb),('grad', grd),('logi', logi),('mlp', mlp),('svm', svm)])
vote.fit(x_train, y_train)

VotingClassifier(estimators=[('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None, gamma=None,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=Non...
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=None,
                                            reg_alpha=None, reg_lambda=None, ...)),
                             ('grad', GradientBoostingClassifier()),
                             ('logi', LogisticRegression()),
                             ('mlp',
                              MLPClassifier(max_iter=300, random_state=47)),
                             ('svm', SVC())])

In [67]:
train_pred = vote.predict(x_train)
test_pred = vote.predict(x_test)
print()
print(accuracy_score(train_pred,y_train))
print('-x-'*20)
print(accuracy_score(test_pred,y_test))


0.49731369017380195
-x--x--x--x--x--x--x--x--x--x--x--x--x--x--x--x--x--x--x--x-
0.4853448275862069


In [23]:
import lightgbm as lgb

In [56]:
light = MLPClassifier(random_state=47, max_iter=300)
light.fit(x_train, y_train)

MLPClassifier(max_iter=300, random_state=47)

In [57]:
train_pred = light.predict(x_train)
test_pred = light.predict(x_test)
print()
print(accuracy_score(train_pred,y_train))
print('-x-'*20)
print(accuracy_score(test_pred,y_test))


0.4940654874613218
-x--x--x--x--x--x--x--x--x--x--x--x--x--x--x--x--x--x--x--x-
0.47737068965517243


In [70]:
def data_prep(df):
    df['Temperature (C)'] = np.where(df['Temperature (C)']=='-','-10',df['Temperature (C)'])
    lis = df['Temperature (C)']
    new = []
    for i in lis:
        new.append(float(i))
    df['Temperature (C)'] = new
    x = df.drop(['Day'],axis=1)
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import classification_report
    from sklearn.preprocessing import StandardScaler
    scale = StandardScaler()
    x[['Apparent Temperature (C)','Humidity','Wind Speed (km/h)','Wind Bearing (degrees)',
                             'Visibility (km)',
                            'Pressure (millibars)','Solar irradiance intensity','Temperature (C)']]=scale.fit_transform(x[['Apparent Temperature (C)','Humidity','Wind Speed (km/h)','Wind Bearing (degrees)',
                             'Visibility (km)',
                            'Pressure (millibars)','Solar irradiance intensity','Temperature (C)']])
    x = pd.get_dummies(x)
    return x

In [71]:
test = pd.read_csv('test_CloudCondition.csv')
x= data_prep(test)
trial01 = vote.predict(x)
trial01 = encoder.inverse_transform(trial01)

In [72]:
day = pd.read_csv('test_CloudCondition.csv')
temp = pd.DataFrame()
temp['Day'] = day['Day']
temp['Cloud_Condition'] = trial01
temp.head()
temp.to_csv('trial_FINAL_voteMLP_Chinmay.csv',index=False)